In [1]:
from gensim import corpora, models,similarities
import re
import numpy as np
import pandas as pd
import math
import pickle

In [2]:
stopwords=["अंदर","अत","अदि","अप","अपना","अपनि","अपनी","अपने","अभि","अभी","आदि","आप","इंहिं","इंहें","इंहों","इतयादि","इत्यादि","इन","इनका","इन्हीं","इन्हें","इन्हों","इस","इसका","इसकि","इसकी","इसके","इसमें","इसि","इसी","इसे","उंहिं","उंहें","उंहों","उन","उनका","उनकि","उनकी","उनके","उनको","उन्हीं","उन्हें","उन्हों","उस","उसके","उसि","उसी","उसे","एक","एवं","एस","एसे","ऐसे","ओर","और","कइ","कई","कर","करता","करते","करना","करने","करें","कहते","कहा","का","काफि","काफ़ी","कि","किंहें","किंहों","कितना","किन्हें","किन्हों","किया","किर","किस","किसि","किसी","किसे","की","कुछ","कुल","के","को","कोइ","कोई","कोन","कोनसा","कौन","कौनसा","गया","घर","जब","जहाँ","जहां","जा","जिंहें","जिंहों","जितना","जिधर","जिन","जिन्हें","जिन्हों","जिस","जिसे","जीधर","जेसा","जेसे","जैसा","जैसे","जो","तक","तब","तरह","तिंहें","तिंहों","तिन","तिन्हें","तिन्हों","तिस","तिसे","तो","था","थि","थी","थे","दबारा","दवारा","दिया","दुसरा","दुसरे","दूसरे","दो","द्वारा","न","नहिं","नहीं","ना","निचे","निहायत","नीचे","ने","पर","पहले","पुरा","पूरा","पे","फिर","बनि","बनी","बहि","बही","बहुत","बाद","बाला","बिलकुल","भि","भितर","भी","भीतर","मगर","मानो","मे","में","यदि","यह","यहाँ","यहां","यहि","यही","या","यिह","ये","रखें","रवासा","रहा","रहे","ऱ्वासा","लिए","लिये","लेकिन","व","वगेरह","वरग","वर्ग","वह","वहाँ","वहां","वहिं","वहीं","वाले","वुह","वे","वग़ैरह","संग","सकता","सकते","सबसे","सभि","सभी","साथ","साबुत","साभ","सारा","से","सो","हि","ही","हुअ","हुआ","हुइ","हुई","हुए","हे","हें","है","हैं","हो","होता","होति","होती","होते","होना","होने"]
alphabets=['ऄ','अ','आ','इ','ई','उ','ऊ','ऋ','ऌ','ऍ','ऎ','ए','ऐ','ऑ','ऒ','ओ','औ','क','ख','ग','घ','ङ','च','छ','ज','झ','ञ','ट',
'ठ','ड','ढ','ण','त','थ','द','ध','न','ऩ','प','फ','ब','भ','म','य','र','ऱ','ल','ळ','ऴ','व','श','ष','स','ह']

matra=['ऀ','ँ','ं','ः','ऺ','ऻ','़','ा','ि','ी','ु','ू','ृ','ॄ','ॅ','ॆ','े','ै','ॉ','ॊ','ो','ौ','्','ॎ','ॏ','ॕ','ॖ','ॗ']

In [3]:
data=pd.read_csv('../hindi.csv')

In [4]:
def remove_special_characters(text):
    string=''
    for i in text:
        if i in alphabets+matra:
            string=string+i
        else:
            string=string+" "
    return " ".join(string.split())

In [5]:
lsi =  models.LsiModel.load('lsi.model')

In [6]:
dictionary = corpora.Dictionary.load('dictionary')
corpus = corpora.MmCorpus('corpus')

In [7]:
def search(query,counter=10):
    text = remove_special_characters(query)
    words = text.split()
    words = [word for word in words if word not in stopwords]
    vec_bow = dictionary.doc2bow(words)
    vec_lsi = lsi[vec_bow]
    index = similarities.MatrixSimilarity(lsi[corpus],num_features=30)
    scoring = index[vec_lsi]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [9]:
query='मैं किसी को मारना चाहता'

In [10]:
verse_index=search(query)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
359,9,27,यत्करोषि यदश्नासि यज्जुहोषि ददासि यत्।\n\nयत्त...,पिछले श्लोक में श्रीकृष्ण ने कहा कि सभी पदार्थ...
590,17,2,श्री भगवानुवाचत्रिविधा भवति श्रद्धा देहिनां सा...,कोई भी व्यक्ति श्रद्धा विहीन नहीं हो सकता क्यो...
677,18,61,ईश्वरः सर्वभूतानां हृद्देशेऽर्जुन तिष्ठति।भ्रा...,आत्मा की भगवान पर निर्भरता पर बल देते हुए श्री...
95,2,54,अर्जुन उवाच\n\nस्थितप्रज्ञस्य का भाषा समाधिस्थ...,स्थितप्रज्ञस्य (स्थिर बुद्धियुक्त मनुष्य) और स...
547,15,3,न रूपमस्येह तथोपलभ्यते नान्तो न चादिर्न च स...,देहधारी जीवात्माएँ 'संसार' में डूबी रहती हैं अ...
548,15,4,ततः पदं तत्परिमार्गितव्य यस्मिन्गता न निवर्...,देहधारी जीवात्माएँ 'संसार' में डूबी रहती हैं अ...
286,7,12,ये चैव सात्त्विका भावा राजसास्तामसाश्च ये।\n\n...,पिछले चार श्लोकों में अपनी महिमा का वर्णन करने...
94,2,53,श्रुतिविप्रतिपन्ना ते यदा स्थास्यति निश्चला।\n...,"जब साधक आध्यात्मिक मार्ग में उन्नति करते हैं,..."
68,2,27,जातस्य हि ध्रुवो मृत्युर्ध्रुवं जन्म मृतस्य च।...,अंग्रेजी भाषा में एक कहावत प्रचलित है कि 'ऐज श...
101,2,60,यततो ह्यपि कौन्तेय पुरुषस्य विपश्चितः।\n\nइन्द...,इन्द्रियाँ उस अनियंत्रित जंगली घोड़े के समान ह...


In [13]:
data.loc[677]['Commentary']

'आत्मा की भगवान पर निर्भरता पर बल देते हुए श्रीकृष्ण कहते हैं-"अर्जुन! भले ही तुम मेरी आज्ञा का पालन करो या न करो, तुम्हारी स्थिति सदैव मेरे प्रभुत्व में रहेगी। जिस शरीर में तुम रहते हो वह यंत्र मेरी माया शक्ति से निर्मित है। तुम्हारे पूर्व जन्मों के अनुसार मैंने तुम्हारी पात्रता के अनुसार तुम्हें शरीर प्रदान किया है। मैं इसमें स्थित रहता हूँ और तुम्हारे विचारों, शब्दों, और कर्मों का लेखा-जोखा रखता हूँ। इस प्रकार से वर्तमान में जो कर्म तुम करते हो उसका आंकलन करते हुए मैं तुम्हारे भविष्य का निर्णय करता हूँ। यह मत सोचो कि तुम मुझसे किसी भी प्रकार से स्वतंत्र हो। इसलिए अर्जुन तुम्हारे हित में यही है कि तुम मेरी शरण ग्रहण करो।'